In [1]:
# 패키지 불러오기

library(httr)    # 온라인상의 데이터를 가져오기 위한 통신 기능 패키지
library(rjson)   # JSON 처리 패키지
library(dplyr)   # 데이터 정제 패키지
library(ggplot2) # 데이터 정제 패키지
library(stringr)


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang


In [2]:
test <- read.csv("./find_data.csv", fileEncoding = "utf-8", stringsAsFactors = F)

In [3]:
str(test)

'data.frame':	280 obs. of  4 variables:
 $ 영화명  : chr  "7급 공무원" "검은 집" "브레인웨이브" "차형사" ...
 $ 스크린수: int  475 332 1 433 1486 806 51 1278 299 707 ...
 $ 개봉일  : chr  "2009-04-22" "2007-06-21" "2006-08-31" "2012-05-30" ...
 $ 감독    : chr  "신태라" "신태라" "신태라" "신태라" ...


In [4]:
test$개봉일 <- str_replace_all(test$개봉일,"-","")

In [5]:
test

영화명,스크린수,개봉일,감독
7급 공무원,475,20090422,신태라
검은 집,332,20070621,신태라
브레인웨이브,1,20060831,신태라
차형사,433,20120530,신태라
82년생 김지영,1486,20191023,김도영
감기,806,20130814,김성수
무명인,51,20140529,김성수
아수라,1278,20160928,김성수
야수,299,20060112,김성수
강철중: 공공의 적 1-1,707,20080619,강우석


In [47]:
# 영화 진흥원에서 발급받은 API키
kobis_api_key = "fd5d23725f8b05cc91cbca3848ac7437"

# 조회 시작 개봉 연도
targetDt = test$개봉일[1]
multiMovieYn = "N"
itemPerPage = "10"
repNationCd = "K" # 한국 코드

In [7]:
# 총 개수를 가져오기 위한 변수 선언
temp_api_url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s&multiMovieYn=%s&itemPerPage=%s&repNationCd=%s"
temp_url <- sprintf(temp_api_url, kobis_api_key, targetDt, multiMovieYn, itemPerPage, repNationCd)
temp_resp <- GET(temp_url)
temp_resp_dat <- content(temp_resp, as="parse", encoding="utf-8")

In [8]:
result <- c()
for (i in 1:length(temp_resp_dat$boxOfficeResult$dailyBoxOfficeList)){
    if(temp_resp_dat$boxOfficeResult$dailyBoxOfficeList[[i]]$movieNm == test$영화명[1]) {
        result <- rbind(result, cbind(영화명=test$영화명[1],cbind(초기스크린수=temp_resp_dat$boxOfficeResult$dailyBoxOfficeList[[i]]$scrnCnt, 일일관객수=temp_resp_dat$boxOfficeResult$dailyBoxOfficeList[[i]]$audiCnt)))
    }
}

In [9]:
result

영화명,초기스크린수,일일관객수
7급 공무원,339,43513


In [55]:
g_data <- c()
for (j in 1:nrow(test)) {
    targetDt = test$개봉일[j]
    # 총 개수를 가져오기 위한 변수 선언
    temp_api_url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s&multiMovieYn=%s&itemPerPage=%s&repNationCd=%s"
    temp_url <- sprintf(temp_api_url, kobis_api_key, targetDt, multiMovieYn, itemPerPage, repNationCd)
    temp_resp <- GET(temp_url)
    temp_resp_dat <- content(temp_resp, as="parse", encoding="utf-8")
    for (i in 1:length(temp_resp_dat$boxOfficeResult$dailyBoxOfficeList)) {
        g_data <- rbind(g_data,as.data.frame(t(unlist(temp_resp_dat$boxOfficeResult$dailyBoxOfficeList[[i]]))))
    }    
}


ERROR: Error in t.default(unlist(temp_resp_dat$boxOfficeResult$dailyBoxOfficeList[[i]])): 인자가 행렬을 가지지 않습니다


In [53]:
g_data

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt
1,1,1,2,OLD,20081737,7급 공무원,2009-04-22,276274500,62.8,269160500,3783.5,557160500,43513,41998,2772.1,87235,339,1786
2,2,2,-1,OLD,20090156,그림자 살인,2009-04-02,145102000,33.0,-22956500,-13.7,10640627500,22385,-3891,-14.8,1593536,275,1495
3,3,3,-1,OLD,20090193,우리 집에 왜 왔니,2009-04-09,16647000,3.8,-8880000,-34.8,1322699000,2579,-1406,-35.3,203859,169,403
4,4,4,0,OLD,20090169,실종,2009-03-19,2063500,0.5,15500,0.8,4268787000,475,8,1.7,625023,12,26
5,5,5,2,OLD,20081672,과속스캔들,2008-12-03,28000,0.0,-5000,-15.2,53797930400,4,-1,-20,8222429,1,1
6,6,6,-1,OLD,20090085,슬픔보다 더 슬픈 이야기,2009-03-11,10000,0.0,-50500,-83.5,4724208000,2,-8,-80,715799,1,1
7,7,7,0,NEW,20090042,핸드폰,2009-02-19,0,0.0,0,100,4221938000,0,0,100,622486,1,1
8,8,7,0,NEW,20081716,쌍화점,2008-12-30,0,0.0,0,100,25411823400,0,0,100,3749034,1,1
9,9,7,1,OLD,20090225,인사동 스캔들,2009-04-29,0,0.0,0,100,12794000,0,0,100,2029,11,11
10,1,1,0,OLD,20070205,검은 집,2007-06-21,436174500,82.9,271407000,164.7,610336000,70811,46223,188,96741,307,1901


In [36]:
test[6,]

,영화명,스크린수,개봉일,감독
6,감기,806,20130814,김성수


In [42]:
for (j in 1:6) {
    targetDt = test$개봉일[j]
    # 총 개수를 가져오기 위한 변수 선언
    temp_api_url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s&multiMovieYn=%s&itemPerPage=%s&repNationCd=%s"
    temp_url <- sprintf(temp_api_url, kobis_api_key, targetDt, multiMovieYn, itemPerPage, repNationCd)
    temp_resp <- GET(temp_url)
    temp_resp_dat <- content(temp_resp, as="parse", encoding="utf-8")
    for (i in 1:length(temp_resp_dat$boxOfficeResult$dailyBoxOfficeList)) {
        g_data <- rbind(g_data,data.frame(t(unlist(temp_resp_dat$boxOfficeResult$dailyBoxOfficeList[[i]]))))
    }    
}

In [52]:
    targetDt = test$개봉일[6]
    # 총 개수를 가져오기 위한 변수 선언
    temp_api_url = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s&multiMovieYn=%s&itemPerPage=%s&repNationCd=%s"
    temp_url <- sprintf(temp_api_url, kobis_api_key, targetDt, multiMovieYn, itemPerPage, repNationCd)
    temp_resp <- GET(temp_url)
    temp_resp_dat <- content(temp_resp, as="parse", encoding="utf-8")
    for (i in 1:length(temp_resp_dat$boxOfficeResult$dailyBoxOfficeList)) {
        g_data <- rbind(g_data,as.data.frame(t(unlist(temp_resp_dat$boxOfficeResult$dailyBoxOfficeList[[i]]))))
    }    

In [41]:
temp_resp_dat$boxOfficeResult$dailyBoxOfficeList

[[1]]
[[1]]$rnum
[1] "1"

[[1]]$rank
[1] "1"

[[1]]$rankInten
[1] "4"

[[1]]$rankOldAndNew
[1] "OLD"

[[1]]$movieCd
[1] "20124937"

[[1]]$movieNm
[1] "감기"

[[1]]$openDt
[1] "2013-08-14"

[[1]]$salesAmt
[1] "2012379500"

[[1]]$salesShare
[1] "32.1"

[[1]]$salesInten
[1] "2006996500"

[[1]]$salesChange
[1] "37284"

[[1]]$salesAcc
[1] "2959473500"

[[1]]$audiCnt
[1] "304661"

[[1]]$audiInten
[1] "303956"

[[1]]$audiChange
[1] "43114.3"

[[1]]$audiAcc
[1] "431461"

[[1]]$scrnCnt
[1] "671"

[[1]]$showCnt
[1] "3577"


[[2]]
[[2]]$rnum
[1] "2"

[[2]]$rank
[1] "2"

[[2]]$rankInten
[1] "6"

[[2]]$rankOldAndNew
[1] "OLD"

[[2]]$movieCd
[1] "20136049"

[[2]]$movieNm
[1] "숨바꼭질"

[[2]]$openDt
[1] "2013-08-14"

[[2]]$salesAmt
[1] "1973544000"

[[2]]$salesShare
[1] "31.5"

[[2]]$salesInten
[1] "1971108000"

[[2]]$salesChange
[1] "80915.8"

[[2]]$salesAcc
[1] "2096471000"

[[2]]$audiCnt
[1] "293917"

[[2]]$audiInten
[1] "293511"

[[2]]$audiChange
[1] "72293.3"

[[2]]$audiAcc
[1] "311907"

[[2]]$scrnCnt
[1] "623"

[[2]]$showCnt
[1] "3377"


[[3]]
[[3]]$rnum
[1] "3"

[[3]]$rank
[1] "3"

[[3]]$rankInten
[1] "-2"

[[3]]$rankOldAndNew
[1] "OLD"

[[3]]$movieCd
[1] "20126674"

[[3]]$movieNm
[1] "설국열차"

[[3]]$openDt
[1] "2013-08-01"

[[3]]$salesAmt
[1] "1373881000"

[[3]]$salesShare
[1] "21.9"

[[3]]$salesInten
[1] "-237819000"

[[3]]$salesChange
[1] "-14.8"

[[3]]$salesAcc
[1] "51118945500"

[[3]]$audiCnt
[1] "199925"

[[3]]$audiInten
[1] "-35575"

[[3]]$audiChange
[1] "-15.1"

[[3]]$audiAcc
[1] "7145121"

[[3]]$scrnCnt
[1] "635"

[[3]]$showCnt
[1] "3071"


[[4]]
[[4]]$rnum
[1] "4"

[[4]]$rank
[1] "4"

[[4]]$rankInten
[1] "-2"

[[4]]$rankOldAndNew
[1] "OLD"

[[4]]$movieCd
[1] "20136052"

[[4]]$movieNm
[1] "더 테러 라이브"

[[4]]$openDt
[1] "2013-07-31"

[[4]]$salesAmt
[1] "911317000"

[[4]]$salesShare
[1] "14.5"

[[4]]$salesInten
[1] "-288592500"

[[4]]$salesChange
[1] "-24.1"

[[4]]$salesAcc
[1] "30844250891"

[[4]]$audiCnt
[1] "133313"

[[4]]$audiInten
[1] "-42450"

[[4]]$audiChange
[1] "-24.2"

[[4]]$audiAcc
[1] "4323366"

[[4]]$scrnCnt
[1] "504"

[[4]]$showCnt
[1] "2748"


[[5]]
[[5]]$rnum
[1] "5"

[[5]]$rank
[1] "5"

[[5]]$rankInten
[1] "2"

[[5]]$rankOldAndNew
[1] "OLD"

[[5]]$movieCd
[1] "20131304"

[[5]]$movieNm
[1] "은밀하게 위대하게 (확장판)"

[[5]]$openDt
[1] "2013-08-05"

[[5]]$salesAmt
[1] "1438000"

[[5]]$salesShare
[1] "0.0"

[[5]]$salesInten
[1] "-1298000"

[[5]]$salesChange
[1] "-47.4"

[[5]]$salesAcc
[1] "25099500"

[[5]]$audiCnt
[1] "204"

[[5]]$audiInten
[1] "-212"

[[5]]$audiChange
[1] "-51"

[[5]]$audiAcc
[1] "3466"

[[5]]$scrnCnt
[1] "3"

[[5]]$showCnt
[1] "7"


[[6]]
[[6]]$rnum
[1] "6"

[[6]]$rank
[1] "6"

[[6]]$rankInten
[1] "0"

[[6]]$rankOldAndNew
[1] "NEW"

[[6]]$movieCd
[1] "20130662"

[[6]]$movieNm
[1] "가자, 장미여관으로"

[[6]]$openDt
[1] "2013-08-14"

[[6]]$salesAmt
[1] "380000"

[[6]]$salesShare
[1] "0.0"

[[6]]$salesInten
[1] "380000"

[[6]]$salesChange
[1] "100"

[[6]]$salesAcc
[1] "2276000"

[[6]]$audiCnt
[1] "180"

[[6]]$audiInten
[1] "180"

[[6]]$audiChange
[1] "100"

[[6]]$audiAcc
[1] "496"

[[6]]$scrnCnt
[1] "2"

[[6]]$showCnt
[1] "2"


[[7]]
[[7]]$rnum
[1] "7"

[[7]]$rank
[1] "7"

[[7]]$rankInten
[1] "0"

[[7]]$rankOldAndNew
[1] "NEW"

[[7]]$movieCd
[1] "20130243"

[[7]]$movieNm
[1] "허풍"

[[7]]$openDt
[1] "2013-08-14"

[[7]]$salesAmt
[1] "468000"

[[7]]$salesShare
[1] "0.0"

[[7]]$salesInten
[1] "468000"

[[7]]$salesChange
[1] "100"

[[7]]$salesAcc
[1] "468000"

[[7]]$audiCnt
[1] "106"

[[7]]$audiInten
[1] "106"

[[7]]$audiChange
[1] "100"

[[7]]$audiAcc
[1] "106"

[[7]]$scrnCnt
[1] "3"

[[7]]$showCnt
[1] "3"


[[8]]
[[8]]$rnum
[1] "8"

[[8]]$rank
[1] "8"

[[8]]$rankInten
[1] "-4"

[[8]]$rankOldAndNew
[1] "OLD"

[[8]]$movieCd
[1] "20124082"

[[8]]$movieNm
[1] "미스터 고"

[[8]]$openDt
[1] "2013-07-17"

[[8]]$salesAmt
[1] "438000"

[[8]]$salesShare
[1] "0.0"

[[8]]$salesInten
[1] "-10396500"

[[8]]$salesChange
[1] "-96"

[[8]]$salesAcc
[1] "9324173000"

[[8]]$audiCnt
[1] "69"

[[8]]$audiInten
[1] "-1757"

[[8]]$audiChange
[1] "-96.2"

[[

In [15]:
nrow(test)

[1] 280

In [16]:
nrow(g_data)
g_data

[1] 28

rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt
1,1,2,OLD,20081737,7급 공무원,2009-04-22,276274500,62.8,269160500,3783.5,557160500,43513,41998,2772.1,87235,339,1786
2,2,-1,OLD,20090156,그림자 살인,2009-04-02,145102000,33.0,-22956500,-13.7,10640627500,22385,-3891,-14.8,1593536,275,1495
3,3,-1,OLD,20090193,우리 집에 왜 왔니,2009-04-09,16647000,3.8,-8880000,-34.8,1322699000,2579,-1406,-35.3,203859,169,403
4,4,0,OLD,20090169,실종,2009-03-19,2063500,0.5,15500,0.8,4268787000,475,8,1.7,625023,12,26
5,5,2,OLD,20081672,과속스캔들,2008-12-03,28000,0.0,-5000,-15.2,53797930400,4,-1,-20,8222429,1,1
6,6,-1,OLD,20090085,슬픔보다 더 슬픈 이야기,2009-03-11,10000,0.0,-50500,-83.5,4724208000,2,-8,-80,715799,1,1
7,7,0,NEW,20090042,핸드폰,2009-02-19,0,0.0,0,100,4221938000,0,0,100,622486,1,1
8,7,0,NEW,20081716,쌍화점,2008-12-30,0,0.0,0,100,25411823400,0,0,100,3749034,1,1
9,7,1,OLD,20090225,인사동 스캔들,2009-04-29,0,0.0,0,100,12794000,0,0,100,2029,11,11
1,1,2,OLD,20081737,7급 공무원,2009-04-22,276274500,62.8,269160500,3783.5,557160500,43513,41998,2772.1,87235,339,1786


In [ ]:
temp_resp_dat

In [ ]:
test$개봉일[1]